In [7]:
from fundus_vessels_toolkit.pipelines import FundusVesselSegToGraph
from fundus_vessels_toolkit.segment import segment_vessels, clear_gpu_cache
from fundus_vessels_toolkit.segment_to_graph import populate_geometry
from fundus_vessels_toolkit.utils.bezier import BezierCubic, BSpline, fit_bezier_cubic
from fundus_vessels_toolkit.utils.graph.measures import extract_branch_geometry
from fundus_vessels_toolkit.utils.image import load_img
from fundus_vessels_toolkit.utils.jppype import Mosaic

#### Load image

In [8]:
# Path to the raw fundus image
RAW_PATH = "/home/gaby/These/Data/Fundus/Vessels/FIVES/test/downsampled-1024/1-images/30_A.jpg"

# Load the raw image
raw = load_img(RAW_PATH)

# Segment the vessels
vessels = segment_vessels(raw)

# Clear the GPU cache
clear_gpu_cache()

#### Parse the vascular graph

In [9]:
max_vessel_diameter = raw.shape[1] // 50
seg2graph = FundusVesselSegToGraph(max_vessel_diameter)
graph = seg2graph(vessels)

#### Parse geometry

In [10]:
graph = populate_geometry(graph, vessels)

#### Display the graph

In [13]:
mosaic = Mosaic(3, raw, 800)
mosaic.add_label("vessel", vessels)

mosaic[0]["vgraph"] = graph.jppype_layer(edge_map=False, edge_labels=True, node_labels=False)
mosaic[1]["vgraph"] = graph.jppype_layer(edge_map=True, edge_labels=False, node_labels=True)
mosaic[2]["vgraph"] = graph.jppype_layer(edge_map=False, edge_labels=True, node_labels=False, bspline_attr="bspline")
mosaic[2]["tangent"] = graph.geometric_data().jppype_bspline_tangents("bspline", 0.4)

mosaic.show()

GridBox(children=(View2D(linkedTransformGroup='2acd004192a5431cb58bc3774db07bdb'), View2D(linkedTransformGroup…

#### Extract bifurcation infos

In [15]:
from fundus_vessels_toolkit.utils.graph.vascular_tree import extract_bifurcations_parameters

geo_data = graph.geometric_data()
calibres = geo_data.branch_data("calibre")
tangents = geo_data.branch_data("tangents")
extract_bifurcations_parameters(calibres, tangents, graph.branch_list, False)

,branch0,branch1,branch2,theta1,theta2,c0,c1,c2
nodeID,,,,,,,,
5,3,4,2,121.208023,179.999981,3.079360,1.935267,2.417354
18,15,13,14,21.282210,70.171341,6.955388,5.808674,5.311000
20,17,14,18,52.156062,128.637488,5.958255,5.469267,3.826958
22,20,10,19,35.277392,39.240156,8.125817,4.866677,5.601011
23,21,22,18,10.003302,50.120655,4.718815,2.522790,3.079147
30,26,16,12,12.925234,15.688035,9.494900,7.851784,9.206785
39,31,15,24,-13.203493,110.733956,11.012324,10.552984,5.720349
42,30,32,35,16.421605,59.766229,9.001856,6.769902,4.980939
44,38,36,37,35.521477,55.192453,5.202022,4.782782,3.821833
